Libraries Used

In [2]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import keras_metrics
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
#STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [1]:
!pip install keras_metrics

Training data which has been cleaned before

In [3]:

olidData=pd.read_csv('training_data.tsv', sep='\t')

olidData.text=olidData.text.astype(str)
olidData

,id,text,actual_label
0,1157097496465825792,artists use clip studio paint please dm looks ...,NOT
1,1159951246964547584,personORGillORGworkORGORGmightORGneedORGpens,NOT
2,1161854610727997440,person 1 mycareer trailer huh,NOT
3,1187418039719026689,aint settling piece shit ass hoes,"OFF, UNT"
4,1157363369898328064,person person yea 100 part group,NOT
...,...,...,...
132246,1155703226819330048,person separated 3 books like 912 chapters fin...,NOT
132247,1161585527650492416,person im sure everyone shit story moz police,"OFF, UNT"
132248,1161209043970473984,person person give shit wield power use power ...,"OFF, TIN, GRP"
132249,1160153244087267328,person children love shit,"OFF, UNT"


Tokenizing and preparing for model

In [4]:
maxWords = 50000
maxLength = 250
embedDimension = 100

tokenizer = Tokenizer(num_words=maxWords, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(olidData['text'])
indexedWords = tokenizer.word_index
print('Found %s unique tokens.' % len(indexedWords))


X_train = tokenizer.texts_to_sequences(olidData['text'].values)
X_train = pad_sequences(X_train, maxlen=maxLength)
print('Shape of data tensor:', X_train.shape)

Y_train = pd.get_dummies(olidData['actual_label'])
print('Shape of label tensor:', Y_train.shape)

print(X_train.shape,Y_train.shape)

Found 76231 unique tokens.
Shape of data tensor: (132251, 250)
Shape of label tensor: (132251, 5)
(132251, 250) (132251, 5)


Neural Network model

In [22]:
neuralNetworkModel = Sequential()
neuralNetworkModel.add(Dense(500, activation='relu', input_dim=250))
neuralNetworkModel.add(Dense(100, activation='relu'))
neuralNetworkModel.add(Dense(5, activation='softmax'))

# Compile the model
neuralNetworkModel.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy' ,keras_metrics.precision(),keras_metrics.recall(),keras_metrics.f1_score(),keras_metrics.false_positive(),keras_metrics.false_negative()])
print(neuralNetworkModel.summary())
history=neuralNetworkModel.fit(X_train, Y_train, epochs=5)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 500)               125500    
_________________________________________________________________
dense_13 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 505       
Total params: 176,105
Trainable params: 176,105
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
132251/132251 [==============================] - 19s 143us/step - loss: 7.2438 - acc: 0.5344 - precision: 0.5707 - recall: 0.8562 - f1_score: 0.6849 - false_positive: 48274.0000 - false_negative: 10776.0000
Epoch 2/5
132251/132251 [==============================] - 19s 144us/step - loss: 6.9841 - acc: 0.5526 - precision: 0.5731 - recall: 0.9131 - f1_score: 0.70

Test Data being read

In [7]:
testData=pd.read_csv('olid-training-v1.0.tsv', sep='\t')
#olidData=pd.read_csv('training_data.tsv', sep='\t')
testData['actual_label'] = testData[['subtask_a', 'subtask_b', 'subtask_c']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)
testData.tweet=testData.tweet.astype(str)
testData

,id,tweet,subtask_a,subtask_b,subtask_c,actual_label
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,"OFF, UNT"
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,"OFF, TIN, IND"
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,NOT
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,"OFF, UNT"
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,NOT
...,...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND,"OFF, TIN, IND"
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN,NOT
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH,"OFF, TIN, OTH"
13238,27429,@USER Pussy,OFF,UNT,NaN,"OFF, UNT"


Code to clean test data

In [8]:
nlp = spacy.load("en_core_web_sm")
olidData = olidData.reset_index(drop=True)
re_space = re.compile('[/(){}\[\]\|@,;]')
re_symbols = re.compile('[^0-9a-z #+_]')
stop_words = set(stopwords.words('english'))
re_user=re.compile('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)')

def clean_text(text):
    text=re_user.sub('Person', text)
    lower = text.lower()
    space_replace = re_space.sub(" ",lower)
    nosymb = re_symbols.sub("",space_replace)
    text = re.sub(r"\s*\b(?:{})\b".format("|".join(stop_words)), "", nosymb)
    doc = nlp(text)
    for entity in doc.ents:
      if entity.label_=="PERSON" or entity.label_=="NORP" or entity.label_=="ORG" or entity.label_=="GPE":
          text=text.replace(entity.text, entity.label_)
    return re.sub(r" {2,}", " ", text.strip())

testData['tweet'] = testData['tweet'].apply(clean_text)
testData

,id,tweet,subtask_a,subtask_b,subtask_c,actual_label
0,86426,person ask native americans take,OFF,UNT,NaN,"OFF, UNT"
1,90194,person person go home youre drunk person #maga...,OFF,TIN,IND,"OFF, TIN, IND"
2,16820,amazon investigating chinese employees selling...,NOT,NaN,NaN,NOT
3,62688,person someone shouldvetaken piece shit volcano,OFF,UNT,NaN,"OFF, UNT"
4,43605,person person obama wanted liberals amp illega...,NOT,NaN,NaN,NOT
...,...,...,...,...,...,...
13235,95338,person sometimes get strong vibes people mans ...,OFF,TIN,IND,"OFF, TIN, IND"
13236,67210,benidorm creamfields maga shabby summer,NOT,NaN,NaN,NOT
13237,82921,person report garbage dont give crap,OFF,TIN,OTH,"OFF, TIN, OTH"
13238,27429,person pussy,OFF,UNT,NaN,"OFF, UNT"


Preparing and tokenizing test data for the model

In [9]:
tokenizer = Tokenizer(num_words=maxWords, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(testData['tweet'])
indexedWords = tokenizer.word_index
print('Found %s unique tokens.' % len(indexedWords))


X_test = tokenizer.texts_to_sequences(testData['tweet'].values)
X_test = pad_sequences(X_test, maxlen=maxLength)
print('Shape of data tensor:', X_test.shape)

Y_test = pd.get_dummies(testData['actual_label'])
print('Shape of label tensor:', Y_test.shape)

Found 20373 unique tokens.
Shape of data tensor: (13240, 250)
Shape of label tensor: (13240, 5)


Evaluation of test data and returning Loss,Accuracy,Precision,Recall and FScore for test data.

In [26]:
accr = neuralNetworkModel.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Precision: {:0.3f}\n  Recall: {:0.3f}\n  F1Score: {:0.3f}'.format(accr[0],accr[1],accr[2],accr[3],accr[4]))

13240/13240 [==============================] - 1s 44us/step
Test set
  Loss: 5.943
  Accuracy: 0.629
  Precision: 0.665
  Recall: 0.938
  F1Score: 0.778


In [0]:
import numpy as np

def InsertPredictions(test_data,model):
    lstNot=[]
    lstOffGrp=[]
    lstOffInd=[]
    lstOffOth=[]
    lstOffUnt=[]
    lst=["NOT","OFF, TIN, GRP","OFF, TIN, IND","OFF, TIN, OTH","OFF, UNT"]
    dd=testData
    lst_text = dd['tweet'].tolist()
    y_prob = model.predict(test_data)
    for prob in y_prob:
      lstNot.append(prob[0])
      lstOffGrp.append(prob[1])
      lstOffInd.append(prob[2])
      lstOffOth.append(prob[3])
      lstOffUnt.append(prob[4])
    y_classes = y_prob.argmax(axis=-1)
    lst_classes=y_classes.tolist()
    lst_predictions = [lst[i] for i in lst_classes]
    lst_original=dd['actual_label'].tolist()
    lst_prob=y_prob.tolist()
    df = pd.DataFrame(lst_text,columns =['tweet'])
    df['Original']=lst_original
    df['Predictions']=lst_predictions
    df['Not Off.']=lstNot
    df['Off. Group']=lstOffGrp
    df['Off. Ind.']=lstOffInd
    df['Off. Others']=lstOffOth
    df['Off. Untargeted']=lstOffUnt
    lstSummary=[]
    for index, row in df.iterrows():
      if row['Original']==row['Predictions']:
        lstSummary.append(1)
      else:
        lstSummary.append(0)
    df['IsCorrect']=lstSummary
    return df

predicted=InsertPredictions(X_test,neuralNetworkModel)
predicted


In [0]:
def calScores(frame):
  lst=['NOT','OFF, TIN, GRP','OFF, TIN, IND','OFF, TIN, OTH','OFF, UNT']
  lstFscore=[]
  for item in lst:
    tp=len(frame[(frame['IsCorrect']==1) & (frame['Predictions']==item)])
    fp=len(frame[(frame['IsCorrect']==0) & (frame['Predictions']==item)])
    #precision=tp/tp+fp
    precision=tp/(tp+fp)
    print("Precision for "+item+" : "+str(precision))
    fn=len(frame[(frame['Original']==item) & (frame['Predictions']!=item)])
    recall=tp/(tp+fn)
    print("Recall for "+item+" : "+str(recall))
    fScore=(2*precision*recall)/(precision+recall)
    lstFscore.append(fScore)
  return lstFscore


lstFscore=calScores(predicted)
